In [1]:
import pandas as pd
import os
import FinanceDataReader as fdr
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

## 데이터셋 확인

In [2]:
path = './data'
list_name = 'Stock_List.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list.sort_values(by='종목코드',ascending=True)

,종목명,종목코드,상장시장
109,메리츠화재,000060,KOSPI
126,하이트진로,000080,KOSPI
67,유한양행,000100,KOSPI
69,CJ대한통운,000120,KOSPI
156,두산,000150,KOSPI
...,...,...,...
328,넥스틴,348210,KOSDAQ
31,하이브,352820,KOSPI
199,솔브레인,357780,KOSDAQ
176,티와이홀딩스,363280,KOSPI


In [3]:
start_date = '20210104'
end_date = '20211029'
sample_code = '005930'
stock = fdr.DataReader(sample_code, start = start_date, end = end_date)
stock

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-01-04,81000,84400,80200,83000,38655276,0.024691
2021-01-05,81600,83900,81600,83900,35335669,0.010843
2021-01-06,83300,84500,82100,82200,42089013,-0.020262
2021-01-07,82800,84200,82700,82900,32644642,0.008516
2021-01-08,83300,90000,83000,88800,59013307,0.071170
...,...,...,...,...,...,...
2021-10-25,69900,70600,69500,70200,10029621,-0.002841
2021-10-26,70600,71500,70400,71100,10528252,0.012821
2021-10-27,71000,71000,70000,70100,10295316,-0.014065


In [17]:
stock.isnull().sum()

Open      0
High      0
Low       0
Close     0
Volume    0
Change    0
dtype: int64

In [19]:
stock.dtypes

Open        int64
High        int64
Low         int64
Close       int64
Volume      int64
Change    float64
dtype: object

## submission 종목 코드 확인

In [4]:
list_name = 'sample_submission.csv'
submission = pd.read_csv(os.path.join(path,list_name))
target_cols = list(submission.columns)[1:]

## input data 준비

In [5]:
stock_close = np.array(stock['Close'])
stock_x = stock_close[:-2]
stock_y = stock_close[1:-1]

## 모델 준비(Linear Regression, RandomForestRegressor)

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
lin_reg = LinearRegression()
rf_reg = RandomForestRegressor()

In [23]:
# #kfold
# from sklearn.model_selection import TimeSeriesSplit
# ts_split = TimeSeriesSplit(n_splits=5)
# cv_accuracy = []

In [53]:
stock_x_tmp = stock_x.reshape(-1, 1)
stock_y_tmp = stock_y.reshape(-1, 1)

array([70100], dtype=int64)

In [54]:
lin_reg.fit(stock_x_tmp, stock_y_tmp)
x_test = stock_x[-1]
pred = lin_reg.predict(x_test.reshape(-1,1))
y_test = stock_y[-1]
print(f'pred : {pred}, real : {y_test}')

pred : [[70247.08434995]], real : 70700


In [57]:
november = []
stock_close = np.array(stock['Close'])
for i in range(25):
    stock_x = stock_close[:-2].reshape(-1,1)
    stock_y = stock_close[1:-1].reshape(-1,1)

    x_test = stock_x[-1]

    lin_reg.fit(stock_x, stock_y)
    pred = lin_reg.predict(x_test.reshape(-1,1))
    november.append(pred)

    stock_close = np.append(stock_close, pred)
    stock_close = np.delete(stock_close, 0)
    

In [58]:
november

[array([[70247.08434995]]),
 array([[70808.35752998]]),
 array([[69928.03372004]]),
 array([[70379.30162585]]),
 array([[70912.9047969]]),
 array([[70099.7623098]]),
 array([[70548.37509275]]),
 array([[71035.68606617]]),
 array([[70257.17471164]]),
 array([[70679.56929828]]),
 array([[71102.54672288]]),
 array([[70360.96376166]]),
 array([[70786.7035744]]),
 array([[71195.31381912]]),
 array([[70463.84504098]]),
 array([[70915.89201183]]),
 array([[71264.09201442]]),
 array([[70545.45739332]]),
 array([[70981.38288394]]),
 array([[71304.14146629]]),
 array([[70604.11227139]]),
 array([[71042.78579409]]),
 array([[71349.17444627]]),
 array([[70655.85636139]]),
 array([[71094.9795298]])]

In [26]:
stock_test = fdr.DataReader(sample_code, start = '20210917', end = '20210924')
stock_test # 장이 열리지 않은 날은 아얘 데이터가 없다 -> isnull로 못 찾음

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-09-17,16950,17350,16700,17300,412972,0.026706
2021-09-23,19450,22200,18650,20450,10153308,0.182081
2021-09-24,20850,21850,19650,20050,3656760,-0.019560


In [8]:
# NMAE
from sklearn.metrics import mean_absolute_error

def nmae(code, y_test, y_pred):
    mae = mean_absolute_error(y_test, y_pred)
    nmae_denominator = np.sum(abs(y_test))/y_test.shape[0]
    accuracy = mae / nmae_denominator * 100
    # print(f'{code} score : {accuracy}')
    return accuracy

In [10]:
# xgboost
import xgboost
xgb_model = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, max_depth = 7, gamma=0, subsample=0.75, colsample_bytree=0.75, reg_lambda=0.8, reg_alpha=0.8)

In [ ]:
def normalize(stock_close):
    # StandardScaler()...fit_transform...
    return stock_close

def denormalize(stock_close):
    return stock_close

# Main

In [11]:
# xgb
_tqdm = tqdm(target_cols)
for code in _tqdm:
    _tqdm.set_description(f"[CODE : {code}]")
    sample_code = code
    stock = fdr.DataReader(code, start = start_date, end = end_date)

    november = []
    stock_close = np.array(stock['Close'])
    stock_close = normalize(stock_close)
    for i in range(5):
        stock_x = stock_close[-61:-2].reshape(-1,1)
        stock_y = stock_close[-60:-1].reshape(-1,1)

        x_test = stock_x[-1]

        xgb_model.fit(stock_x, stock_y)
        pred = xgb_model.predict(x_test.reshape(-1,1))
        november.append(pred)

        stock_close = np.append(stock_close, pred)
        stock_close = np.delete(stock_close, 0)
    # november_target = november[:5] + november[-5:]
    november_target = november + november
    november_last = list(map(lambda x:x[0], november_target))
    submission.loc[:, code] = november_last

    

[CODE : 950130]: 100%|██████████| 370/370 [02:29<00:00,  2.47it/s]


In [ ]:
# randomforest? sgd? elasticnet? lstm?
_tqdm = tqdm(target_cols)
for code in _tqdm:
    _tqdm.set_description(f"[CODE : {code}]")
    sample_code = code
    stock = fdr.DataReader(code, start = start_date, end = end_date)

    november = []
    stock_close = np.array(stock['Close'])
    for i in range(5):
        stock_x = stock_close[-61:-2].reshape(-1,1)
        stock_y = stock_close[-60:-1].reshape(-1,1)

        x_test = stock_x[-1]

        rf_reg.fit(stock_x, stock_y)
        pred = rf_reg.predict(x_test.reshape(-1,1))
        november.append(pred)

        stock_close = np.append(stock_close, pred)
        stock_close = np.delete(stock_close, 0)
    # november_target = november[:5] + november[-5:]
    november_target = november + november
    november_last = list(map(lambda x:x[0], november_target))
    submission.loc[:, code] = november_last

    

[CODE : 950130]: 100%|██████████| 370/370 [02:29<00:00,  2.47it/s]


In [12]:
# performance_check
answer = pd.read_csv('./data/answer_submission.csv')
_tqdm = tqdm(target_cols)
nmae_score = 0
total_score = 0
for code in _tqdm:
    _tqdm.set_description(f"[CODE : {code}]")
    y_pred = submission[code][:5]
    y_answer = answer[code][:5]
    nmae_score += nmae(code, y_answer, y_pred)

total_score = nmae_score / 370
print(f'total_score = {total_score}')

[CODE : 950130]: 100%|██████████| 370/370 [00:00<00:00, 677.02it/s]

total_score = 3.352146924913583


In [13]:
# code = '000060'
# sample_value = [i for i in range(10)]
# submission.loc[:,code] = sample_value
submission.to_csv('./submission/xgbregressor_v2.csv', index=False)

In [25]:
def compare(answer_path: os.path, my_answer_path: os.path):
    answer = pd.read_csv(answer_path)
    my_answer = pd.read_csv(my_answer_path)
    
    columns = answer.columns
    format = pd.DataFrame(np.zeros((1, 371)), index=["NMAE"], columns=columns)
    format = format.rename(columns={'Day': 'Total'})

    for code in answer.columns[1:]:
        true = answer[code].to_numpy(dtype=np.float32)
        pred = my_answer[code].to_numpy(dtype=np.float32)
        normalized_error = np.abs(true - pred) / (true)
        normalized_mean_error = np.mean(normalized_error)

        format[code][0] = normalized_mean_error * 100

    total_nmae = np.mean(format, axis=1)
    format['Total'] = total_nmae
    result = format.T.copy(deep=True)
    result.reset_index(inplace=True)
    
    result[1:] = result[1:].sort_values(["NMAE"], ascending=False, axis=0)
    result.to_csv("./results.csv", index=False)

compare('./data/answer_submission.csv', './submission/xgbregressor_v1.csv')

In [25]:
# start_date = '20210104'
# end_date = '20211029'
# # for code in target_cols:
# # sample_code = code
# # stock = fdr.DataReader(sample_code, start = start_date, end = end_date)
# # stock_x = stock.drop('Close', axis=1).to_numpy()
# # stock_y = stock['Close'].copy().to_numpy()

# n_iter = 0
# for train_index, test_index in ts_split.split(stock_x):
#     x_train, x_test = stock_x[train_index], stock_x[test_index]
#     y_train, y_test = stock_y[train_index], stock_y[test_index]
#     lin_reg.fit(x_train, y_train)
#     valid_pred = lin_reg.predict(x_test)
#     n_iter += 1
    # mae = mean_absolute_error(y_test, valid_pred)
    # nmae_denominator = np.sum(abs(y_test))/y_test.shape[0]
    # accuracy = mae / nmae_denominator * 100
#     print(f'{n_iter} 교차검증 정확도 : {accuracy}, 학습 데이터 크기 : {x_train.shape[0]}, 검증 데이터 크기 : {x_test.shape[0]}')
#     cv_accuracy.append(accuracy)
# print('\n')
# print(f'평균 검증 정확도 : {np.mean(cv_accuracy)}')


1 교차검증 정확도 : 1.251811949496452, 학습 데이터 크기 : 34, 검증 데이터 크기 : 34
2 교차검증 정확도 : 1.8019715580925881, 학습 데이터 크기 : 68, 검증 데이터 크기 : 34
3 교차검증 정확도 : 1.1782601044670427, 학습 데이터 크기 : 102, 검증 데이터 크기 : 34
4 교차검증 정확도 : 1.4519986673279472, 학습 데이터 크기 : 136, 검증 데이터 크기 : 34
5 교차검증 정확도 : 0.9687740125626234, 학습 데이터 크기 : 170, 검증 데이터 크기 : 34


평균 검증 정확도 : 0.880763931731957
